In [1]:
import json
import re
from pathlib import Path

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================
INPUT_DIR = Path('data/json/llm_ready')
OUTPUT_DIR = Path('data/json/cleaned_factors')

def singularizar_texto(texto):
    """
    Aplica regras básicas de singularização do português palavra por palavra.
    Evita depender de bibliotecas externas pesadas (como spacy/nltk).
    """
    palavras = texto.split()
    palavras_singulares = []

    # Lista de exceções comuns que terminam em 's' mas são singulares ou invariáveis
    excecoes = {'ônibus', 'lápis', 'cais', 'mês', 'gás', 'país', 'simples', 'atrás', 'após', 'através', 'férias'}

    for p in palavras:
        original = p
        # Regras de sufixo (ordem importa)
        if p in excecoes:
            pass
        elif p.endswith('ns'): # jovens -> jovem, itens -> item
            p = p[:-2] + 'm'
        elif p.endswith('res'): # fatores -> fator, melhores -> melhor
            p = p[:-2]
        elif p.endswith('zes'): # vezes -> vez
            p = p[:-2] + 'z'
        elif p.endswith('ões'): # ações -> ação
            p = p[:-3] + 'ão'
        elif p.endswith('ais'): # locais -> local
            p = p[:-3] + 'al'
        elif p.endswith('éis') or p.endswith('eis'): # papéis -> papel, niveis -> nivel
            p = p[:-3] + 'el'
        elif p.endswith('s') and not p.endswith('ss'): # indeterminados -> indeterminado, humanos -> humano
            p = p[:-1]

        palavras_singulares.append(p)

    return " ".join(palavras_singulares)

def padronizar_frases(texto):
    """
    Realiza substituições de termos específicos (correção de preposições).
    Deve ser rodado APÓS o texto estar em lowercase e singular.
    """
    correcoes = {
        "aplicação de comando": "aplicação do comando",
        "aplicação no comando": "aplicação do comando",
        "manutenção de aeronave": "manutenção da aeronave",
        "planejamento de voo": "planejamento do voo"
    }

    for errado, correto in correcoes.items():
        if errado in texto:
            texto = texto.replace(errado, correto)

    return texto

def limpar_fatores_contribuintes(dados_fatores):
    """
    Recebe a seção 'fatores_contribuintes', limpa caracteres estranhos,
    remove conectivos falhos, normaliza tudo para lowercase e singular.
    """
    # 1. Unificação: Garante que temos uma única string para trabalhar
    texto_bruto = ""
    if isinstance(dados_fatores, list):
        texto_bruto = " ".join(dados_fatores)
    elif isinstance(dados_fatores, str):
        texto_bruto = dados_fatores
    else:
        return []

    # 2. Normalização inicial
    texto_bruto = re.sub(r'\s+', ' ', texto_bruto).strip()

    # 3. Estratégia de Divisão (Split por ponto e vírgula)
    itens_brutos = re.split(r';', texto_bruto)

    fatores_limpos = []

    for item in itens_brutos:
        item = item.strip()

        if not item:
            continue

        # 4. Limpeza Cirúrgica (Regex)

        # Passo A (ATUALIZADO): Remove conectivos "e -", "e, -", "e. -"
        # Adicionamos [.,]? para aceitar pontuação após o "e"
        item = re.sub(r'^\s*(?:e[.,]?\s+)?-?\s*', '', item, flags=re.IGNORECASE)

        # Passo B: Remove marcadores de lista comuns
        item = re.sub(r'^[\•\*\.]+\s*', '', item)

        # Passo C: Remove ponto final se estiver no fim da string
        item = re.sub(r'\.$', '', item)

        # Passo D: Remove espaços nas pontas novamente
        item = item.strip()

        # Passo E (NOVO): Normaliza para Lowercase
        # Transforma "Supervisão Gerencial" em "supervisão gerencial"
        item = item.lower()

        # Passo F (NOVO): Converte para Singular
        # "indeterminados" -> "indeterminado", "fatores humanos" -> "fator humano"
        item = singularizar_texto(item)

        # Passo G (NOVO): Padronização Específica (Correções pontuais)
        item = padronizar_frases(item)

        # Só adiciona se sobrou texto útil
        if len(item) > 2:
            fatores_limpos.append(item)

    # 5. Fallback
    # Se a lista ficou vazia ou quase vazia, tenta estratégia agressiva (hífen)
    if len(fatores_limpos) <= 1 and len(texto_bruto) > 50 and ';' not in texto_bruto:
        itens_alternativos = re.split(r'\s+-\s+', texto_bruto)

        # Aplicamos a mesma lógica de limpeza e .lower() aqui também
        fatores_limpos = []
        for x in itens_alternativos:
            limpo = re.sub(r'\.$', '', x.strip()).lower()
            limpo = singularizar_texto(limpo) # Aplica singular aqui também
            limpo = padronizar_frases(limpo)  # Aplica correções específicas
            if len(limpo) > 2:
                fatores_limpos.append(limpo)

    return fatores_limpos

def processar_diretorio():
    if not INPUT_DIR.exists():
        print(f"Diretório não encontrado: {INPUT_DIR}")
        # Cria pastas dummy para teste se não existirem, apenas para não dar erro no exemplo
        INPUT_DIR.mkdir(parents=True, exist_ok=True)
        print(f"Criado diretório vazio para teste: {INPUT_DIR}")
        return

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    arquivos = list(INPUT_DIR.glob("*.json"))
    print(f"Iniciando limpeza em {len(arquivos)} arquivos...")

    processados = 0

    for arquivo in arquivos:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)

            if 'conteudo' in dados and 'fatores_contribuintes' in dados['conteudo']:
                fatores_originais = dados['conteudo']['fatores_contribuintes']

                # Executa a limpeza
                fatores_novos = limpar_fatores_contribuintes(fatores_originais)

                dados['conteudo']['fatores_contribuintes'] = fatores_novos

                caminho_saida = OUTPUT_DIR / arquivo.name
                with open(caminho_saida, 'w', encoding='utf-8') as f_out:
                    json.dump(dados, f_out, ensure_ascii=False, indent=4)

                processados += 1

        except Exception as e:
            print(f"Erro no arquivo {arquivo.name}: {e}")

    print(f"Concluído! {processados} arquivos limpos salvos em '{OUTPUT_DIR}'.")

if __name__ == "__main__":
    # Bloco de teste rápido para validar a lógica sem arquivos
    print("--- Teste de Lógica ---")
    exemplo_sujo = "Aplicação de comando; Manutenção de aeronaves; planejamento de voo."
    print(f"Entrada: {exemplo_sujo}")
    print(f"Saída:   {limpar_fatores_contribuintes(exemplo_sujo)}")
    print("-----------------------")

    # Executa o processamento real
    processar_diretorio()

--- Teste de Lógica ---
Entrada: Aplicação de comando; Manutenção de aeronaves; planejamento de voo.
Saída:   ['aplicação do comando', 'manutenção da aeronave', 'planejamento do voo']
-----------------------
Iniciando limpeza em 1567 arquivos...
Concluído! 1567 arquivos limpos salvos em 'data\json\cleaned_factors'.
